In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('credit_batch_1.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,isFraud
0,88672.0,-3.859881,2.632881,-5.264265,3.446113,-0.675231,-1.904959,-3.291041,-0.985766,-1.168114,...,1.664119,0.785075,0.068412,0.778961,-0.863166,-0.006810,-1.065734,1.773326,1.18,1
1,62059.0,-1.644403,3.129852,-2.576977,3.415573,-0.448525,-1.241893,-1.991652,1.002665,-2.809071,...,0.417762,-0.648576,-0.318617,-0.680413,0.389869,0.055750,0.394682,0.298821,6.62,1
2,13323.0,-5.454362,8.287421,-12.752811,8.594342,-3.106002,-3.179949,-9.252794,4.245062,-6.329801,...,1.846165,-0.267172,-0.310804,-1.201685,1.352176,0.608425,1.574715,0.808725,1.00,1
3,93965.0,-11.397727,7.763953,-18.572307,6.711855,-10.174216,-4.395918,-15.893788,2.083013,-4.988837,...,0.339007,1.342923,0.239217,0.534644,-0.174965,-0.500240,-1.722060,-0.574339,11.40,1
4,52934.0,1.036639,0.407227,0.757706,3.161821,-0.568122,0.202181,-0.689804,0.411380,0.336769,...,-0.050108,0.123761,-0.132568,0.350231,0.507701,0.189621,0.061016,0.063141,0.76,1


In [3]:
data['isFraud'].value_counts()

isFraud
0    8000
1     490
Name: count, dtype: int64

## Split Dataset

In [4]:
X = data.drop(['isFraud'], axis=1)
y = data['isFraud']

#Standardization
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [5]:
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE(random_state=75).fit_resample(X_train, y_train)

pd.Series(y_train).value_counts()

isFraud
0    6405
1    6405
Name: count, dtype: int64

## Define TF Model Layers

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score

2023-05-02 20:16:18.962200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# DNN
model = Sequential([
Dense(input_dim = 30, units = 128, activation = "relu"),
Dense(units= 64, activation = "relu"),
Dropout(0.2),
Dense(units= 32, activation = "relu"),
Dropout(0.2),
Dense(units= 16, activation = "relu"),
Dropout(0.2),
Dense(units=1, activation = "sigmoid")])

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

## Model Training

In [9]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

model.fit(X_train, y_train, epochs=50, batch_size=100)

Epoch 1/50
129/129 [==============================] - 1s 952us/step - loss: 0.2350 - accuracy: 0.9080
Epoch 2/50
129/129 [==============================] - 0s 935us/step - loss: 0.0836 - accuracy: 0.9699
Epoch 3/50
129/129 [==============================] - 0s 891us/step - loss: 0.0473 - accuracy: 0.9830
Epoch 4/50
129/129 [==============================] - 0s 896us/step - loss: 0.0415 - accuracy: 0.9886
Epoch 5/50
129/129 [==============================] - 0s 904us/step - loss: 0.0270 - accuracy: 0.9920
Epoch 6/50
129/129 [==============================] - 0s 943us/step - loss: 0.0186 - accuracy: 0.9959
Epoch 7/50
129/129 [==============================] - 0s 909us/step - loss: 0.0120 - accuracy: 0.9966
Epoch 8/50
129/129 [==============================] - 0s 906us/step - loss: 0.0113 - accuracy: 0.9970
Epoch 9/50
129/129 [==============================] - 0s 912us/step - loss: 0.0078 - accuracy: 0.9984
Epoch 10/50
129/129 [==============================] - 0s 882us/step - loss: 0.008

## Model Evaluation

In [10]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

54/54 [==============================] - 0s 546us/step


In [11]:
accuracy_score(y_test, y_hat)

0.9882214369846879

In [12]:
model.evaluate(X_test, y_test)

54/54 [==============================] - 0s 615us/step - loss: 0.1813 - accuracy: 0.9882


[0.1813000738620758, 0.9882214665412903]